In [ ]:
using Pkg
Pkg.activate("../.")

In [2]:
using Turing
using Stheno
using RDatasets
using Random; Random.seed!(5)
using Plots
using Statistics

# Gaussian process classification
## Introduction
In classification problems, we map features $x$ to it's label $c$, we can decompose this process as two functions $f$ and $g$, $f$ is some nonlinear function that transforms $x$ to middle variables $z$, and $g$ is the sigmoid function that maps $z$ to probability of taking a particular class.

We can summarise the binary classification problem as:
$$
\begin{gather}
z=f(x)\\
p=\sigma(z)\\
c=\left\{\begin{array}{rl}
    1 & \text{if } p>0.5\\
    0 & \text{if } p\le0.5
    \end{array}\right.
\end{gather}
$$

We can assign $f$ a Gaussian process prior, and model the likelihood function $p(c|z)$ by Bernoulli distribution, the overall probability model can be writen as:
$$
p(c|x) = \int p(c|f, x)p(f)\mathbb{d}f
$$
but unlike regression case, the integral over $f$ is intractable, therefore, to perform parameter estimation and inference we need to use Monte Carlo method.

In [3]:
# data preparation
crabs = dataset("MASS","crabs")
crabs = crabs[shuffle(1:size(crabs, 1)), :]
train = crabs[1:div(end,2), :]
test = crabs[div(end,2)+1:end, :]

train_y = Array{Bool}(undef,size(train, 1))
train_y[train.Sp.=="B"].=0
train_y[train.Sp.=="O"].=1
train_x = Matrix(transpose(convert(Array,train[:,4:end])))

test_y = Array{Bool}(undef, size(test, 1))
test_y[test.Sp.=="B"].=0
test_y[test.Sp.=="O"].=1
test_x = Matrix(transpose(convert(Array, test[:, 4:end])));

In [4]:
@show size(train_x)
@show size(test_x)

size(train_x) = (5, 100)
size(test_x) = (5, 100)


(5, 100)

In [87]:
σ(x) = 1.0 / (1.0+exp(-x))

function build_gp(logl, σ², X)
    ard_eq_kernel = σ² * stretch(EQ(), exp.(-logl))
    gp = GP(ard_eq_kernel, GPC())
    prior = gp(ColVecs(X), 1e-6)
    gp, prior
end

# model that learn θ and latent variable f
@model gpc_learn(X, y) = begin
    logl ~ Normal(0.0, 2.0)
    _, prior = build_gp(logl, 1.0, X)
    f ~ prior
    for i in eachindex(y)
        y[i] ~ Bernoulli(σ(f[i]))
    end
end

gpc_learn (generic function with 3 methods)

In [106]:
# inference model (using the MAP)
function gpc_infer(x, logl, Xtrain, fsamples)
    nsamples = size(fsamples, 2)
    fxs = []
    for i in 1:nsamples
        gp, prior = build_gp(logl[i], 1.0, Xtrain)
        new_gp = gp | Obs(prior, fsamples[:, i])
        posterior = new_gp(ColVecs(x))
        push!(fxs, mean(posterior))
    end
    fx_mean = vec(mean(hcat(fxs...), dims=2))
    p = σ.(fx_mean)
    y = Int.(p .> 0.5)
    y
end

gpc_infer (generic function with 1 method)

In [114]:
model = gpc_learn(train_x, train_y)
mcmc_samples = sample(model, HMC(0.01, 10), 5000);

Sampling 100%|███████████████████████████████| Time: 0:49:47


In [116]:
logl_df = mcmc_samples[:logl]
logl = vec(logl_df.value.data)
logl = Array{Float64}(logl)
reserve_logl = logl[1001:end]
@show size(reserve_logl)
@show eltype(reserve_logl)

size(reserve_logl) = (4000,)
eltype(reserve_logl) = Float64


Float64

In [117]:
fsamples_df = mcmc_samples[:f]
fsamples = Matrix(transpose(dropdims(fsamples_df.value.data, dims=3)))
fsamples = convert.(Float64, fsamples)
reserve_fsamples = fsamples[:, 1001:end]
@show size(reserve_fsamples)
@show eltype(reserve_fsamples)

size(reserve_fsamples) = (100, 4000)
eltype(reserve_fsamples) = Float64


Float64

In [118]:
pred_y = gpc_infer(test_x, reserve_logl, train_x, reserve_fsamples);

In [119]:
hcat(pred_y, test_y)

100×2 Array{Int64,2}:
 0  0
 1  0
 1  1
 1  0
 1  1
 1  1
 0  0
 0  0
 1  1
 0  0
 1  1
 1  1
 1  1
 ⋮   
 1  1
 0  0
 1  1
 0  0
 0  0
 1  1
 0  1
 0  0
 0  0
 1  1
 0  0
 0  0

In [120]:
# accuracy function
function accuracy(pred_y, y)
    N = length(y)
    N_neq = sum(abs.(pred_y .- y))
    r = 1.0 - N_neq / N
    r
end

accuracy(pred_y, test_y)

0.89